# Day 2 - Afternoon Session

We will cover

1. Model deployment, Streamlit, saving models, etc.
2. Fine tuning pre-trained models using a GPU
3. Exercise 

# Distributing Your Work

There are several reasons why you might want to make a model you have trained available.

## Publishing / Distributing Models

Once you have trained a model, and you wish to publish it, what are your options?

First, by publishing, what do we mean. For example, let's say you were working on a dataset here at the University for a department. You create a model based on the data they have provided for you, and you would like to use this model in an application that the doctors can use. 

In this case, how would you create an application for them to use? 

You could create a tradtional application, perhaps with Java, and then install this on their machines. 

This is problematic for several reasons... 

A better approach might be to create a web application. Some advantages are:

- You only need to provide a URL to the end users
- The web application can run internally on the network and not be on the internet
- Updates that you make are instantly distributed
- No installation required by the end user, no issues with administation rights
- Multi-platform: any device with a web browser can access your application

Let's see how we might do that now.

## StreamLit

In this section will see how to use StreamLit, a web-application framework for Python. Web-applications are good ways to distrubute your working models. However, web-development is generally considered to be challenging, and therefore StreamLit is a way to develop web apps in Python.

## Basics

Streamlit allows you to build web applications using pure Python. It is similar to Shiny. 

Streamlit provides a number of controls, such as buttons, sliders, and so on so that you can interact with the application and the data. 

To start a Streamlit app you just run:

```sh
$ streamlit run script.py
```

from the command line. 

Let's write a very simple app now. We will place the following code in a file called `slider.py`:

```python
import streamlit as st

# Write a title and some text to the app:
st.title('Slider App')
st.write('Demo of Streamlit Interaction')

# Create a slider widget:
value = st.slider('Please select a value:', 0, 100, 50)

# Write the value back to the user:
st.write("You selected the value of ", value, ' using the slider.')
```

We use the element `slider()` element to display the slider, with a minimum of 0, a maximum value of 100, and a default value of 50.

Also notice, it is convention to import Streamlit as `st`, which gives you access to all the Streamlit controls.

>See all the controls that Streamlit has available here: <https://docs.streamlit.io/library/api-reference> and there is also a useful Cheat Sheet: <https://docs.streamlit.io/library/cheatsheet> 
>
>On the left hand side, you will see all the types of elements, such as layout elements and data elements, and so on.

Of course, we can apply functions to the data, so we can modify this script to show your selected value squared, by adding the following function to the script:

```python
def squared(x):
    return x**2
```

and then modify the `st.write()` statement to say the following: 

```python
st.write("I'm ", squared(x), "years old, squared.")`
```

or something similar.

Notice how when we made a change to the source code, the web application noticed this and asked if we'd like to reload the script. 

### Design principles

But how does Streamlit update the `value` in the script above? How does it know when to update it? Normally in JavaScript you'd need to write all these event handlers to do this, for example by manually writing `onClick(e)` event handlers that are executed when events occur. This is quite complicated and requires a lot of coding and knowledge of HTML, JavaScript, and how web applications work.

Streamlit works in the following way: when you start an application, the `streamlit` application watches your web application, and any change to any element, such as a slider, re-runs your entire script again. You have to keep this in mind, that the entire script is re-run. 

This might sound time consuming and slow, especially if there is a calculation that is performed when a user moves a slider, but you can cache results so that a calculation only needs to be run once. If a function that is executed when an element is changed, you can use the `@st.cache_data` decorator to tell Streamlit to cache the results.

#### Displaying Data
Streamlit is focussed on data science and machine learning applications. Therefore the displaying of data is one of its strengths. 

Consider the following application:

```python
"""
# Displaying Data
"""

import streamlit as st
import pandas as pd
df = pd.DataFrame({
  'first column': [1, 2, 3, 4],
  'second column': [10, 20, 30, 40]
})

df
```

Let's save this to a file called `table.py` and run it using `streamlit run table.py` from the command line.

As you can see, you get a lot of interactivity for free. You can search rows, expand cells by double clicking on them, and so on.

If you are in doubt how to display an element, just try `st.write()` and most likely it will work.

Also, plotting is supported, much like in `R`/`Shiny`. For example the following draws a simple plot:

```python
import streamlit as st
import numpy as np
import pandas as pd

chart_data = pd.DataFrame(
     np.random.randn(20, 3),
     columns=['a', 'b', 'c'])

st.line_chart(chart_data)
```

We can save this to `plot.py` and see its output.


Note, that if you wanted to draw a plot, you could generate the plot using MatPlotLib or Seaborn, and just use `st.write()` and it will most likely work. 

For example, for a MatPlotLib plot we could say the following:

```python
import matplotlib.pyplot as plt

arr = np.random.normal(1, 1, size=100)
fig, ax = plt.subplots()
ax.hist(arr, bins=20)

st.write(fig)
```

Note that there is actually a `pyplot()` function, so you would probably want to use `st.pyplot(fig)` instead of `st.write(fig)` but this demonstrates the fleixibility of the `st.write()` function.

So, those are the basics of how Streamlit applications work: you write a Python script, and Streamlit handles the updating of the elements by re-running the script each time the user makes a change to any button, slider, and so on. Of course, in our case, we wish to integrate a trained model with a web application, and we will do this now.

## Integtration With a Trained Model

For any models that you might train during your PhD, or you work at the medical university, you will want to be able to use Streamlit to actually deploy a machine learning based application. This can take several forms, for example:

- An application that allows an expert to label images, such as the example above. This application could also train a model as 
- An application that allows interaction with a trained model, for example one that allows a doctor to query the network for a classification based on an image he or she might upload

In the first scenario you are using the application to train your model and to collect data. In the second scenario you are using a trained model that has you have already prepared.

Step one, let's create a new file, let's call it 'model.py' and add the following code to it:

```python
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import pandas as pd
import streamlit as st

# Set title of app
st.title("Image Classifier")
st.write("Demo application for the ATSP course")

# Enable users to upload images for the model to make predictions
uploaded_file = st.file_uploader("Upload your image", type = "jpg")
```

If we run this we see we have a nice file uplaoder, which let's use choose a file and upload it to the webpage. The `st.file_uploader()` function is doing all the hard work, including telling the browser that only JPEG images are allowed, that there is a 200MB limit, and also returns the uploaded image to the script in the form of `uploaded_file`. 

Also, as we can see we now need to import Torch and Torch Vision, as we are going to load a model previously trained using Torch. 

If we wanted to display this file back to the user, it is a simple case of using `st.image()` (note, you can add code to a running app, and Streamlit will notice a change and ask yo if you want to rerun the application):

```python
if uploaded_file is not None:
    # Display image that user uploaded
    image = Image.open(uploaded_file)
    st.image(image, caption = 'Uploaded Image', use_container_width = True)
    st.write("")
```

Notice also that we included this `if` statement. What we are doing is checking if an image exists. If it exists, that is if the user uploaded the image, then display it. Otherwise display nothing. 

Remember, every time a user interacts with an element, the entire script is re-run. Meaning after the user uploads a file, the script is re-run, this time with an image contained in `uploaded_file` and then the script above will execute.

Now that we have our image, we can load up a model and pass the image to the model. For that we will define a function that will be called when a user uploads an image. Let's just call it `predict()`: 

```python
def predict(image):
    # Return top 5 predictions ranked by highest probability.
    # First create a ResNet model
    resnet = models.resnet18(pretrained = True)

    # Transform the input image through resizing, etc. We have seen this several times in the course already.
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean = [0.485, 0.456, 0.406],
            std = [0.229, 0.224, 0.225]
        )])

    # Load the image, apply preprocessing, and make predictions
    img = Image.open(image)
    single_image_batch = torch.unsqueeze(transform(img), 0)
    resnet.eval()
    out = resnet(single_image_batch)

    # Read the classes file, which contains the 1000 classes of ImageNet
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]

    # Return the top 5 predictions, ranked by probability
    prob = torch.nn.functional.softmax(out, dim = 1)[0]
    _, indices = torch.sort(out, descending = True)

    return [(classes[i], prob[i].item()) for i in indices[0][:5]]
```

We can place this anywhere in the code, as it will be executed by Streamlit when the app is loaded.

Now we need to display these predictions, which we will do under the image, where we said `if uploaded_file is not None:` previously:

```python
if uploaded_file is not None:

    # Display image that user uploaded
    image = Image.open(uploaded_file)
    st.image(image, caption = 'Uploaded Image', use_container_width= True)
    st.write("")
    
    # Make the prediction
    labels = predict(uploaded_file)
    
    # Format the labels a bit better
    formatted = []
    for i, v in enumerate(labels):
        formatted.append([i+1, v[0].split(',')[1], v[0].split(',')[0], v[1]])
    
    # Save to a Pandas DataFrame
    df = pd.DataFrame(formatted, columns=['Rank', 'Prediction', 'Class Index', 'Probability'])
    df = df.set_index('Rank')
    
    # Show the DataFrame
    st.dataframe(df)
```

Now we can see the results in a Pandas DataFrame. If we wanted to make the table match the size of the parent element, we could use `st.dataframe(df, use_container_width=True)`. 

---

We should think back to our comic that we saw earlier today:

<img src="./img/task.png" width="400"/>

*Source*: <https://xkcd.com/1425/>

# Fine Tuning

First, we will discuss Google Colab. We will use Colab as we can use a GPU for this task.

Colab can be found here: <https://colab.research.google.com>

Why use Google Colab? 

For image data, we generally require a GPU as neural networks require a lot of computational power. We can use Google Colab to do this, as you can use a GPU for free.

So in the next section, we will use Google Colab to fine-tune a neural network that has been pre-trained 